In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os
import time

In [10]:
# Selenium
driver = webdriver.Chrome()  # 크롬드라이버가 설치되어 있어야 함
driver.get("https://comic.naver.com/webtoon")

In [11]:
# HTML 코드 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [12]:
# 웹툰 제목 리스트
title_list = soup.select("span.ContentTitle__title--e3qXt")

# 이미지 리스트
image_list = soup.select("img.Poster__image--d9XTI")

In [13]:
# 저장 폴더 생성
image_dir = 'webtoon_images'
os.makedirs(image_dir, exist_ok=True)

In [15]:
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import os, re, requests

pattern = r'[\\/:"*?<>|]'
headers = {'User-Agent': 'Mozilla/5.0'}

print(f"수집한 웹툰 수: {len(title_list)}")

for title, image in zip(title_list[:len(image_list)], image_list):
    try:
        print(f"[▶] 제목: {title.text}")
        image_src = image.get('src')

        if not image_src:
            print(" 이미지 주소 없음")
            continue

        img_res = requests.get(image_src, headers=headers)

        # 이미지 객체 만들기
        img = Image.open(BytesIO(img_res.content)).convert("RGB")

        # 안전한 파일명 만들기
        filename = re.sub(pattern, '', title.text)

        # 확장자 결정
        ext = f".{img.format.lower()}" if img.format else ".png"

        # 저장
        img.save(os.path.join(image_dir, filename + ext))
        print(f" 저장 완료: {filename}{ext}")

    except UnidentifiedImageError:
        print(f" 이미지 형식 오류: {title.text}")
    except Exception as e:
        print(f" 예외 발생: {e}")

수집한 웹툰 수: 728
[▶] 제목: 용사생활기록부
 저장 완료: 용사생활기록부.png
[▶] 제목: 전직용병의 검사생활
 저장 완료: 전직용병의 검사생활.png
[▶] 제목: 인간천적
 저장 완료: 인간천적.png
[▶] 제목: 만남어플 중독
 저장 완료: 만남어플 중독.png
[▶] 제목: 참교육
 저장 완료: 참교육.png
[▶] 제목: 환생천마
 저장 완료: 환생천마.png
[▶] 제목: 왕과의 야행
 저장 완료: 왕과의 야행.png
[▶] 제목: 절대검감
 저장 완료: 절대검감.png
[▶] 제목: 똑 닮은 딸
 저장 완료: 똑 닮은 딸.png
[▶] 제목: 백수세끼
 저장 완료: 백수세끼.png
[▶] 제목: 소녀의 세계
 저장 완료: 소녀의 세계.png
[▶] 제목: 마음의소리(였던 것)
 저장 완료: 마음의소리(였던 것).png
[▶] 제목: 귀촌리
 저장 완료: 귀촌리.png
[▶] 제목: 시한부 천재가 살아남는 법
 저장 완료: 시한부 천재가 살아남는 법.png
[▶] 제목: 십이지소녀
 저장 완료: 십이지소녀.png
[▶] 제목: 회귀한 공작가의 막내도련님은 암살자
 저장 완료: 회귀한 공작가의 막내도련님은 암살자.png
[▶] 제목: 버림받은 왕녀의 은밀한 침실
 저장 완료: 버림받은 왕녀의 은밀한 침실.png
[▶] 제목: 천재 타자가 강속구를 숨김
 저장 완료: 천재 타자가 강속구를 숨김.png
[▶] 제목: 신화급 귀속 아이템을 손에 넣었다
 저장 완료: 신화급 귀속 아이템을 손에 넣었다.png
[▶] 제목: 딸들
 저장 완료: 딸들.png
[▶] 제목: 파브르 in 사천당가
 저장 완료: 파브르 in 사천당가.png
[▶] 제목: 이직로그
 저장 완료: 이직로그.png
[▶] 제목: 어린이집 다니는 구나
 저장 완료: 어린이집 다니는 구나.png
[▶] 제목: 아포칼립스에 집을 숨김
 저장 완료: 아포칼립스에 집을 숨김.png
[▶] 제목: 별난식당
 저장 완료: 별난식당.png
[▶] 제목: 무벌전
 저장 완료: 무벌전.png


In [ ]:
import os
import re
import requests
from PIL import Image
from io import BytesIO

pattern = r'[\\/:"*?<>|]'
image_dir = 'webtoon_images'
os.makedirs(image_dir, exist_ok=True)

success_count = 0
fail_count = 0
pattern = r'[\\/:"*?<>|]'
seen_urls = set()

for i, (title_elem, image_elem) in enumerate(zip(title_list, image_list)):
    try:
        title = title_elem.get("title") or title_elem.text.strip()
        img_url = image_elem.get("src")

        if not title or not img_url:
            print(f"[❌] 제목 또는 이미지 없음 (index: {i})")
            fail_count += 1
            continue

        if img_url in seen_urls:
            continue
        seen_urls.add(img_url)

        response = requests.get(img_url, timeout=10)
        response.raise_for_status()

        img = Image.open(BytesIO(response.content))
        filename = re.sub(pattern, '', title) + ".png"
        img.save(os.path.join(image_dir, filename))

        print(f"[✔] 저장 완료: {filename}")
        success_count += 1
    except Exception as e:
        print(f"[❌] 저장 실패: {title if 'title' in locals() else 'Unknown'} - {e}")
        fail_count += 1

print(f"\n▶ 총 시도: {i+1}, 성공: {success_count}, 실패: {fail_count}")


[❌] 저장 실패: 용사생활기록부 - 403 Client Error: Access Denied for url: https://image-comic.pstatic.net/webtoon/840332/thumbnail/titledescimage/objectImage_dd0a5d1e-413f-44af-9d74-cb2c7613f3b8.png
[❌] 저장 실패: 전직용병의 검사생활 - 403 Client Error: Access Denied for url: https://image-comic.pstatic.net/webtoon/840332/thumbnail/titledescimage/frontImage_92375ce2-b5c0-4272-8076-70962f988f08.png
[❌] 저장 실패: 인간천적 - 403 Client Error: Access Denied for url: https://image-comic.pstatic.net/webtoon/840147/thumbnail/titledescimage/objectImage_93b6d37c-d4e5-481f-a2bd-ad30397aae5e.png
[❌] 저장 실패: 만남어플 중독 - 403 Client Error: Access Denied for url: https://image-comic.pstatic.net/webtoon/840147/thumbnail/titledescimage/backImage_e81680c3-d1c2-4d42-8fc6-1d8d97e0f1fa.png
[❌] 저장 실패: 참교육 - 403 Client Error: Access Denied for url: https://image-comic.pstatic.net/webtoon/840147/thumbnail/titledescimage/frontImage_475307a5-61ab-40ba-8e2b-d6dd57c428c9.png
[❌] 저장 실패: 환생천마 - 403 Client Error: Access Denied for url: https://image-